In [ ]:
from datasets import load_dataset
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import random
import numpy
import math

'''
------------------------------------------------------------------------------------------------------------
    STARTER CODE AND FUNCTIONS
------------------------------------------------------------------------------------------------------------
'''
dataset = load_dataset("mnist")

#Getting random 100 sample images of each label from 'mnist' dataset
sample_images = []
for i in range(10):
    cur_digit_images = []
    for train_image in dataset['train']:
        if train_image['label'] == i:
            cur_digit_images.append(train_image)
    random_cur_digit_images = random.sample(cur_digit_images, 100) #Taking random 100 images
    sample_images.extend(random_cur_digit_images)

#Flattening images as 1-D list
flatten_images = []
for image in sample_images:
    flatten_image = []
    image_data = image['image'].getdata()
    for pixel in image_data:
        flatten_image.append(pixel)
    flatten_images.append(flatten_image)

#Converting flatten_images as numpy array and forcing it to use float64 for better precision
flatten_images = numpy.array(flatten_images)
flatten_images = [numpy.ravel(numpy.asarray(x,dtype=numpy.float64)) for x in flatten_images]
flatten_images = numpy.array(flatten_images)

#Centering flatten_images
mean = numpy.mean(flatten_images, axis = 0)
centered_images = flatten_images - mean

#Transposing as we need the each data point to be along a column
centered_images = centered_images.T

print("******** Got the train dataset and Centered it **********\n")


#Function to get eigen values and vectors in decending order
def get_eigen_values_and_vectors(matrix):
    eigen_vals , eigen_vecs = numpy.linalg.eigh(matrix)
    #eigh gives eigen values in ascending order hence inversing them accordingly
    eigen_vals = eigen_vals[::-1]
    eigen_vecs = eigen_vecs[:, ::-1]
    return (eigen_vals, eigen_vecs)

# Calculating the Covariance Matrix
covariance_matrix = numpy.dot(centered_images, centered_images.T) / 1000

# Getting required sorted eigen vectors and eigen values
eigen_values, eigen_vectors = get_eigen_values_and_vectors(covariance_matrix)

print("****** Got eigen values and vectors ******\n")

In [ ]:
'''
------------------------------------------------------------------------------------------------------------
    PART (i)
------------------------------------------------------------------------------------------------------------
'''
total_variance = numpy.sum(eigen_values)

#Writing Variance due to each PC into txt file
with open('principal_component_variance.txt', 'w') as f:
    for i, variance in enumerate(eigen_values):
        #Due to floating point representations some 0's will come as very very small -ve values hence making them 0.
        if variance < 0 :
            variance = 0
        ratio = (variance / total_variance) * 100
        f.write(f"Variance explained by PC{i+1} : {ratio} %\n")
print("Printed the Variance explained by PC's to *principal_component_variance.txt* file\n")

#Plotting images of Principle Components
plt.figure(figsize=(30, 50))
for i in range(150):
    plt.subplot(16, 10, i+1)
    pc_image = eigen_vectors[:, i].reshape(28, 28) # Eigen vectors i.e principle components are arranged column-wise
    plt.imshow(pc_image, cmap='gray')
    plt.title(f'PC {i + 1}')
    plt.axis('off')
plt.subplots_adjust(top=0.96)
plt.suptitle('Top 150 Principal Components Images', fontsize=40)
plt.savefig("PC's_Images.png")
# plt.show()

In [ ]:
'''
----------------------------------------------------------------------------------------------------------------
    PART (ii)
----------------------------------------------------------------------------------------------------------------
'''

# A function to return a random flattened test image
def get_random_test_image():
    test_image = None
    flatten_test_img = []
    label = random.randint(0, 9) #Selecting a random label
    for img in dataset['test']:
        if img['label'] == label:
            test_image = img
            break
    test_image_data = test_image['image'].getdata()
    for pixel in test_image_data:
        flatten_test_img.append(pixel)
    flatten_test_img = numpy.array(flatten_test_img)
    return flatten_test_img




n_components = [10, 30, 50, 130, 200, 300, 500, 784]
num_plots = len(n_components)
fig, axs = plt.subplots(2, 4, figsize=(15, 6))

flatten_test_img = get_random_test_image()
centered_test_img = flatten_test_img - mean

for i, n in enumerate(n_components):
    projected_image = []
    #Getting the projected image using 'n' principle components
    for j in range(n):
        projection_length = (numpy.dot(centered_test_img,eigen_vectors[:, j].T))
        if len(projected_image) == 0:
            projected_image = numpy.dot(projection_length, eigen_vectors[:, j].T)
        else:
            projected_image += numpy.dot(projection_length, eigen_vectors[:, j].T)
    #Getting back the original Image by adding mean
    projected_image = projected_image + mean    
    reconstructed_img = projected_image.reshape(28,28)
    
    # Plotting the reconstructed image
    row = i // 4
    col = i % 4
    axs[row, col].imshow(reconstructed_img, cmap='gray')
    axs[row, col].set_title(f'PCs = {n}')
    axs[row, col].axis('off') 
plt.suptitle("Test Image Reconstruction using various number of PC's")
plt.tight_layout() 
plt.savefig('Reconstructed_image.png')
print("Saved Reconstructed Image by varying number of PC's to file named 'Reconstructed_image.png'\n")
# plt.show()


# Calculating # of PC's to get to 95% of Variance
count = 0
current_sum = 0.0
for item in eigen_values:
    current_sum += item
    count += 1
    if (current_sum/total_variance) >= 0.95:
        break
    else:
        continue

print("Minimum required 'd' to classify the different digits correctly :", count, "\n")

In [ ]:
'''
----------------------------------------------------------------------------------------------------------
    FUNCTIONS FOR PART (iii)
----------------------------------------------------------------------------------------------------------
'''

def get_polynomial_kernel_matrix(d):
    kernel_matrix = numpy.zeros((1000, 1000))
    for i in range(1000):
        for j in range(1000):
            kernel_matrix[i][j] = (1 + numpy.dot(flatten_images[i],flatten_images[j].T)) ** d
    return kernel_matrix

def get_radial_kernel_matrix(sigma):
    kernel_matrix = numpy.zeros((1000, 1000))
    for i in range(1000):
        for j in range(1000):
            kernel_matrix[i][j] = math.exp(-1*(numpy.dot(((flatten_images[i]-flatten_images[j]).T),(flatten_images[i]-flatten_images[j])))/(2*(math.pow(sigma,2))))
    return kernel_matrix

def get_centered_kernel_matrix(kernel_matrix):
    kernel_matrix_rows_sum = numpy.zeros(1000)
    kernel_matrix_total_sum = 0
    for i in range(1000):
        for j in range(1000):
            kernel_matrix_rows_sum[i] += kernel_matrix[i][j]
            kernel_matrix_total_sum += kernel_matrix[i][j]
            
    centered_kernel_matrix = numpy.zeros((1000, 1000))
    for i in range(1000):
        for j in range(1000):
            centered_kernel_matrix[i][j] = kernel_matrix[i][j] - ((kernel_matrix_rows_sum[i])/1000) - ((kernel_matrix_rows_sum[j])/1000) + kernel_matrix_total_sum/1000000 
    
    return centered_kernel_matrix


def get_kernel_matrix_eigen_values_and_vectors(d, poly_or_kernel):
    if poly_or_kernel == 1:
        kernel_matrix = get_polynomial_kernel_matrix(d)
    else:
        kernel_matrix = get_radial_kernel_matrix(d)
    centered_kernel_matrix = get_centered_kernel_matrix(kernel_matrix)
    kernel_matrix_eigen_vals, kernel_matrix_eigen_vecs = get_eigen_values_and_vectors(centered_kernel_matrix)
    return (kernel_matrix_eigen_vals, kernel_matrix_eigen_vecs)


#Function to get projection lengths of first 2 Principle Components
def get_PC1_PC2_lengths(d, poly_or_kernel):
    kernel_matrix_eigen_vals, kernel_matrix_eigen_vecs = get_kernel_matrix_eigen_values_and_vectors(d, poly_or_kernel)
    
    # Calculating alpha_k 
    alpha_k = []
    for i in range(len(kernel_matrix_eigen_vals)):
        root_n_lamda = numpy.sqrt(abs(kernel_matrix_eigen_vals[i]))
        alpha_k_i = numpy.divide(kernel_matrix_eigen_vecs[:, i], root_n_lamda)
        alpha_k.append(alpha_k_i)
    alpha_k = numpy.array(alpha_k)
 
    #Getting the Kernel Matrix
    if poly_or_kernel == 1:
        kernel_matrix = get_polynomial_kernel_matrix(d)
    else:
        kernel_matrix = get_radial_kernel_matrix(d)

    #As kernel function is symmentric
    projection_length_PC1 = []
    for i in range(1000):
        projection_length_PC1.append(numpy.dot(alpha_k[0], kernel_matrix[i]))

    projection_length_PC2 = []
    for i in range(1000):
        projection_length_PC2.append(numpy.dot(alpha_k[1], kernel_matrix[i]))
    
    return (projection_length_PC1, projection_length_PC2) 

In [ ]:
'''
--------------------------------------------------------------------------------------------------------------
    PART (iii)
--------------------------------------------------------------------------------------------------------------
'''

# Ploting using polynomial kernel
d_list = [2, 3, 4]
plt.figure(figsize = (14, 11))
for i,d in enumerate(d_list):
    plt.subplot(2, 2, i+1)
    pc1_length, pc2_length = get_PC1_PC2_lengths(d, 1) # 1 here means polynomial
    plt.scatter(pc1_length, pc2_length, label = '(X, Y)', alpha = 0.8)
    plt.xlabel("X -> Projection length's due to PC1")
    plt.ylabel("Y -> Projection length's due to PC2")
    plt.title(f"Plot for Polynomial kernel function by taking d={d}")
    plt.legend()
plt.savefig('Polynomial_Kernel_Projection.png')
print("Saved the plot of projection of each point in the dataset onto the top-2 components using polynomial kernel to 'Polynomial_kernel_Projection.png'\n")
# plt.show()



# Plotting using radial kernel
sigma_list = [2, 10, 100, 1000]

plt.figure(figsize=(15,16))
for i, sigma in enumerate(sigma_list):
    plt.subplot(2, 2, i+1)
    pc1_length, pc2_length = get_PC1_PC2_lengths(sigma, 2)
    plt.scatter(pc1_length, pc2_length, label = '(X, Y)', alpha = 0.8)
    plt.xlabel("X -> Projection length's due to PC1")
    plt.ylabel("Y -> Projection length's due to PC2")
    plt.title(f"Plot for Radial kernel function by taking sigma={sigma}")
    plt.legend()
plt.savefig('Radial_kernel_Projection.png')
print("Saved the plot of projection of each point in the dataset onto the top-2 components using Radial kernel to 'Radial_kernel_Projection.png'\n")
# plt.show()